In [104]:
import pandas as pd
import re
from pydub.utils import mediainfo
import os

In [105]:
df1 = pd.read_parquet('/data/ruiqi.yan/data/reasoning/gsm8k/main/train-00000-of-00001.parquet')
df2 = pd.read_parquet('/data/ruiqi.yan/data/reasoning/gsm8k/main/test-00000-of-00001.parquet')

df = pd.concat([df1, df2], ignore_index=True)

In [106]:
len(df)

8792

In [107]:
df.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [108]:
df = df.rename(columns={'question': 'source_text'})
df = df.rename(columns={'answer': 'target_text'})

In [109]:
df['id'] = range(1, len(df) + 1)
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]

In [110]:
df['reference'] = df['target_text'].apply(lambda x: x.split('\n')[-1][5:])

In [111]:
df.head()

,id,source_text,target_text,reference
0,1,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,72
1,2,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,10
2,3,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...",5
3,4,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,42
4,5,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,624


In [112]:
def filter_text(text):
    if len(text.split()) > 30: 
        return False
    if not text or not isinstance(text, str):
        return False
    if re.search(r'http[s]?://\S+', text) or re.search(r'\S+@\S+', text):
        return False
    if re.search(r'[\x00-\x1F\x7F]+', text):
        return False
    if re.search(r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]{2,}', text):
        return False
    if re.search(r'[\$:]', text):
        return False
    if re.search(r'[+\-*/^=()]', text):
        return False
    if re.search(r'\b\d*(?:\.\d+)?(ml|cm|kg|m|mg|mm|dm|min|g|l|km|m²|s|h)\b', text):
        return False
    if re.search(r'[\u00BC-\u00BE\u2150-\u218F√±×÷]', text):
        return False
    if re.search(r'[¼½¾⅓⅔⅕⅖⅗⅘⅙⅚⅛⅑⅒√±×÷]', text):
        return False
    return True

In [113]:
df_filtered = df[df['source_text'].apply(filter_text)]

In [114]:
df_filtered.loc[:, 'id'] = range(1, len(df_filtered) + 1)
len(df_filtered)

887

In [115]:
df_filtered.to_json('/data/ruiqi.yan/data/reasoning/gsm8k/test.jsonl', orient='records', lines=True, force_ascii=False)